In [1]:
from distributed import LocalCluster, Client
cluster = LocalCluster()
cluster

LocalCluster(066a35f3, 'tcp://127.0.0.1:32871', workers=4, threads=12, memory=18.56 GiB)

In [2]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 18.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32871,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 18.56 GiB
Comm: tcp://127.0.0.1:36889,Total threads: 3
Dashboard: http://127.0.0.1:34311/status,Memory: 4.64 GiB
Nanny: tcp://127.0.0.1:37977,


In [3]:
from scipy.integrate import quad
import numpy as np
import uproot
from math import pi
from IPython.display import display


# Set parameters
l0 = 2*pi
B0 = 1
R = 0.5*l0

Lx           = 2.*l0
Ly           = 2.*l0
Lz           = 2.*l0

# Resolution
resx         = 16.      #64               # nb of cells in one laser wavelength
resy         = 16.                     
resz         = 16.

#Ring 1
x0 = Lx/2
y0 = Ly/2
z0 = .1*Lz

#Ring 2
x1 = Lx/2
y1 = Ly/2
z1 = .9*Lz

def get_Bx(x, y, z, x0,y0,z0):
    def integrand_x(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.cos(t)/r_3
    a, b = 0, 2*np.pi
    bx = B0*quad(integrand_x, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bx

def get_By(x, y, z, x0,y0,z0):
    def integrand_y(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.sin(t)/r_3
    a, b = 0, 2*np.pi
    by = B0*quad(integrand_y, a, b, args=(x-x0, y-y0, z-z0))[0]
    return by

def get_Bz(x, y, z, x0,y0,z0):
    def integrand_z(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*(R - y*np.sin(t) - x*np.cos(t))/r_3
    a, b = 0, 2*np.pi
    bz = B0*quad(integrand_z, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bz

def two_rings_x(x,y,z):
    Bx_ring1 = get_Bx(x,y,z, x0,y0,z0)
    Bx_ring2 = get_Bx(x,y,z, x1,y1,z1)
    return Bx_ring1 + Bx_ring2

def two_rings_y(x,y,z):
    By_ring1 = get_By(x,y,z, x0,y0,z0)
    By_ring2 = get_By(x,y,z, x1,y1,z1)
    return By_ring1 + By_ring2

def two_rings_z(x,y,z):
    Bz_ring1 = get_Bz(x,y,z, x0,y0,z0)
    Bz_ring2 = get_Bz(x,y,z, x1,y1,z1)
    return Bz_ring1 + Bz_ring2


# Domain
dx           = l0/resx
dy           = l0/resy
dz           = l0/resz

Nx = Lx/dx
Ny = Ly/dy
Nz = Lz/dz

print(f"Lx: {int(Lx)}")
print(f"Nx: {int(Nx)}")

x = np.linspace(0, Lx, int(Nx))
y = np.linspace(0, Ly, int(Ny))
z = np.linspace(0, Lz, int(Nz))

print("Calculating B components.")
Ax = []
Ay = []
Az = []
num = 0
x_index = 0

dh = display(" ",display_id=True)

def int_row_x(i, j, z):
    Cx = []
    for k in z:
            Cx.append(two_rings_x(i,j,k))
    return Cx
def int_row_y(i, j, z):
    Cy = []
    for k in z:
            Cy.append(two_rings_y(i,j,k))
    return Cy
def int_row_z(i, j, z):
    Cz = []
    for k in z:
            Cz.append(two_rings_z(i,j,k))
    return Cz

for i in x:
    Dx = []
    Dy = []
    Dz = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in y:
        Dx.append(client.submit(int_row_x, i,j,z))
        Dy.append(client.submit(int_row_y, i,j,z))
        Dz.append(client.submit(int_row_z, i,j,z))
    Ax.append(Dx)
    Ay.append(Dy)
    Az.append(Dz)   

Lx: 12
Nx: 32
Calculating B components.


'32 out of 32'

In [4]:
Bx = client.gather(Ax)
By = client.gather(Ay)
Bz = client.gather(Az)

In [5]:
with uproot.recreate("B_field_inputs.root") as f:
    f["B_field"] = {"Bx": Bx, "By": By, "Bz": Bz}
    #f["B_field"] = {"Bx": Bx, "Bz": Bz}

In [6]:
np.shape(Bx)

(32, 32, 32)